In [ ]:
from tensorflow import keras
import os
from numba import cuda  # https://stackoverflow.com/a/52354865/6476994
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from readInImages import readInImages
import cv2

In [ ]:
# allows all images to be displayed at once (else only displays the last call to plt.imshow())
# https://stackoverflow.com/a/41210974
def displayImage(image, caption = None, colour = None) -> None:
    plt.figure(figsize=(10,10))
    if(colour != None):
        plt.imshow(image, cmap=colour)
    else:
        plt.imshow(image)
        
    if(caption != None):
        # display caption below picture (https://stackoverflow.com/a/51486361)
        plt.figtext(0.5, 0.01, caption, wrap=True, horizontalalignment='center', fontsize=12)

In [ ]:
model_to_load = 'final/ZFNet-map-baseline-1'
# model_to_load = 'final/ZFNet-map-pre-processing-hist-eq-dark-and-canny-non-empty-1'

# model_to_load = 'final/InceptionV3-map-baseline-1'
# model_to_load = 'final/InceptionV3-map-pre-processing-hist-eq-dark-and-canny-non-empty-1'

# model_to_load = 'final/custom-architecture-map-baseline-1'
# model_to_load = 'final/custom-architecture-map-pre-processing-hist-eq-dark-and-canny-non-empty-1'

In [ ]:
model = keras.models.load_model('saved_models/{}.h5'.format(model_to_load))

## Use a distinct dataset to assess trained model's performance
* still camera trap images from same location(s), but subset was not used for training, validation, or testing
* note: images are not pre-processed, and are simply loaded in and resized
* TODO: use Australian camera trap images from different location(s) to assess performance
* TODO: save image/prediction pairs - will allow inputting many images to classify without having to display them all, which uses a lot of memory

In [ ]:
unlabelled_test_images = []
folders_dataset = next(os.walk('data/unlabelled_test_data'))[1]
folders_dataset = sorted(folders_dataset)
print('folders_dataset: {}'.format(folders_dataset))
all_image_names = []

for folder in folders_dataset:
    unlabelled_test_images = [*unlabelled_test_images, *readInImages('unlabelled_test_data', folder, False)]
    
    curr_folder_image_names = next(os.walk('data/unlabelled_test_data/{}'.format(folder)),
                         (None, None, []))[2]  # [] if no file    
    curr_folder_image_names = sorted(curr_folder_image_names)
    curr_folder_and_image_names = []
    for curr_folder_image_name in curr_folder_image_names:
        curr_folder_and_image_names.append("{}/{}".format(folder, curr_folder_image_name))
    all_image_names = [*all_image_names, *curr_folder_and_image_names]

unlabelled_test_images = np.stack(unlabelled_test_images, axis = 0)
print('stacked unlabelled_test_images shape: {}'.format(unlabelled_test_images.shape))

### Predict

In [ ]:
class_mappings = ['Bird', 'Cat', 'Dog', 'Empty photo', 'Emu', 'Fox', 'Humman Presense/Deployment', 'Kangaroo', 'Other', 'Rabbit']

In [ ]:
predictions_prob = model.predict(unlabelled_test_images)

for i in range(len(unlabelled_test_images)):    
    sorted_prob = np.sort(predictions_prob[i])
    max_pred = sorted_prob[len(sorted_prob) - 1]
    # find the index in the unsorted array, as that maps to an index in `class_mappings`
    max_pred_index = np.where(predictions_prob[i] == max_pred)[0][0]
    
    second_max_pred = sorted_prob[len(sorted_prob) - 2]
    second_max_pred_index = np.where(predictions_prob[i] == second_max_pred)[0][0]
    
    third_max_pred = sorted_prob[len(sorted_prob) - 3]
    third_max_pred_index = np.where(predictions_prob[i] == third_max_pred)[0][0]
    
    pred_str = "Guesses ordered by most probable for image '{}': ".format(all_image_names[i])
    pred_str += "\n(1) '{}' at {}% ".format(class_mappings[max_pred_index], np.around(float(max_pred * 100), 1))
    pred_str += "\n(2) '{}' at {}% ".format(class_mappings[second_max_pred_index], np.around(float(second_max_pred * 100), 1))
    pred_str += "\n(3) '{}' at {}% ".format(class_mappings[third_max_pred_index], np.around(float(third_max_pred * 100), 1))
    
    displayImage(cv2.cvtColor(unlabelled_test_images[i], cv2.COLOR_BGR2RGB), pred_str)

## Free up the GPU's memory

In [ ]:
cuda.select_device(0)
cuda.close()